# This will be an implementation of Grover's Search Algorithm with the intention 

#### Imports

In [5]:
import math
from qiskit import QuantumCircuit
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

### setting up quantum computer

#### video if interested:https://www.youtube.com/watch?v=c30KrWjHaw4

In [6]:
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.least_busy(operational=True, simulator=False)
backend.name

'ibm_sherbrooke'

### Search Algorithm

In [7]:
def grover_oracle(marked_states):

    if not isinstance(marked_states, list):
        marked_states = [marked_states]

    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
   
    for target in marked_states:
        rev_target = target[::-1]
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
    return qc

#### Instance of Grover Search 

In [13]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)
oracle.draw()


┌───┐   ┌───┐
q_0: ──────■─┤ X ├─■─┤ X ├
           │ ├───┤ │ ├───┤
q_1: ──────■─┤ X ├─■─┤ X ├
     ┌───┐ │ ├───┤ │ └───┘
q_2: ┤ X ├─■─┤ X ├─■──────
     └───┘   └───┘

In [15]:
grover_op = GroverOperator(oracle)
grover_op.decompose().draw()

global phase: π
                 ┌───┐   ┌───┐┌───┐┌───┐     ┌───┐┌───┐     
state_0: ──────■─┤ X ├─■─┤ X ├┤ H ├┤ X ├──■──┤ X ├┤ H ├─────
               │ ├───┤ │ ├───┤├───┤├───┤  │  ├───┤├───┤     
state_1: ──────■─┤ X ├─■─┤ X ├┤ H ├┤ X ├──■──┤ X ├┤ H ├─────
         ┌───┐ │ ├───┤ │ ├───┤├───┤├───┤┌─┴─┐├───┤├───┤┌───┐
state_2: ┤ X ├─■─┤ X ├─■─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├
         └───┘   └───┘   └───┘└───┘└───┘└───┘└───┘└───┘└───┘

##### because we are using grover search we want to increase the amplitude of our desired state as much as possible and minimize other states

In [16]:
optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
)

In [17]:
qc = QuantumCircuit(grover_op.num_qubits)
# Initiliazes all to plus state 
qc.h(range(grover_op.num_qubits))

qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
qc.measure_all()
qc.draw()

┌───┐┌────┐ ░ ┌─┐      
   q_0: ┤ H ├┤0   ├─░─┤M├──────
        ├───┤│    │ ░ └╥┘┌─┐   
   q_1: ┤ H ├┤1 Q ├─░──╫─┤M├───
        ├───┤│    │ ░  ║ └╥┘┌─┐
   q_2: ┤ H ├┤2   ├─░──╫──╫─┤M├
        └───┘└────┘ ░  ║  ║ └╥┘
meas: 3/═══════════════╩══╩══╩═
                       0  1  2